In [1]:
import pandas as pd
import datetime
import os
from dateutil import parser
from sqlalchemy import create_engine

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

db_host = 'localhost'
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_port = os.getenv('DB_PORT')

engine = create_engine("postgresql+psycopg2://{user}:{pw}@{host}/{db}".format(host=db_host, db=db_name, user=db_user, pw=db_password))

def find_initial_positive_trend(row,initial_cutoff_percent = 0.4):
    if row['change'] > row['open']*initial_cutoff_percent/100:
        return [True, row['datetime'], None, row['close_previous'], row['close_current'], row['change_percentage'], row['volume'], 1, 0]
    else:
        return [False, None, None, None, None, None, None, 0, 1]
    
def find_positive_trend(row, retracement_threshold = 0.7, stagnation_percent = 0.3, stag_threshold = 10):   

    half = row['half']
    new_half = (row['open'] + row['close'])/2

    if abs(new_half-half) > half*stagnation_percent/100:
        stag_count = 0
        row['half'] = new_half
    else:
        stag_count = row['stag_count'] + 1

    if row['trend'] == True:
        current_change_percentage = round(((row['close']-row['start_price'])/row['start_price'])*100,2)
        trend_change = row['trend_change']
        new_volume = row['trend_volume'] + row['volume']
        if current_change_percentage <= retracement_threshold*trend_change or row['datetime_current'].time() == datetime.time(15, 29, 0) or stag_count > stag_threshold:
            return[False, False, None, None, None, None, None, None, 0, stag_count, row['half']]
        elif current_change_percentage > trend_change:
            return[False, True, row["start_datetime"], row["datetime_current"], row["start_price"], row["close"], current_change_percentage, new_volume, row["trend_candle_count"]+1, stag_count, row['half']]
        elif current_change_percentage > retracement_threshold*trend_change and stag_count <= stag_threshold:
            return[True, True, row["start_datetime"], row["end_datetime"], row["start_price"], row["end_price"], trend_change, row["trend_volume"], row["trend_candle_count"], stag_count, row['half']]
        
    else:
        if row['change'] > row['open']*stagnation_percent/100:
            row['half'] = new_half
            return [False, True, row['datetime_current'], row['datetime_current'], row['open'], row['close'], row['change_percentage'], row['volume'], 1, 0, row['half']]
        else:
            return [False, False, None, None, None, None, None, None, 0, stag_count, row['half']]

day_candles = pd.read_sql_query(f"select * from candle60 where date(datetime) = '2024-07-12' order by datetime", con=engine)
previous_day_candles = pd.read_sql_query(f"select * from candle60 where date(datetime) = '2024-07-11' order by datetime", con=engine)

idx = previous_day_candles.groupby('symbol')['datetime'].idxmax()
max_datetime_per_symbol = previous_day_candles.loc[idx]
previous_candles = max_datetime_per_symbol[["token", "symbol", "close"]].reset_index(drop=True)

distinct_datetimes = day_candles['datetime'].unique()

for distinct_datetime in distinct_datetimes:
    current_candles = day_candles[day_candles['datetime'] == distinct_datetime].reset_index(drop=True)

    print(distinct_datetime)

    if parser.parse(str(distinct_datetime)).time() == datetime.time(9, 15, 0):
        trend_summary = pd.merge(current_candles, previous_candles, how = 'inner', on=['token','symbol'], suffixes=('_current', '_previous'))
        trend_summary['change'] = trend_summary['close_current'] - trend_summary['close_previous']
        trend_summary['change_percentage'] = round((trend_summary['change']/trend_summary['close_previous'])*100,2)
        trend_summary['half'] = (trend_summary['open'] + trend_summary['close_current'])/2
        trend_summary['pause'] = False
        trend_summary['trend_props'] = trend_summary.apply(find_initial_positive_trend, axis=1)
        trend_summary[['trend', 'start_datetime', 'end_datetime', 'start_price', 'end_price', 'trend_change', 'trend_volume', 'trend_candle_count', 'stag_count']] = pd.DataFrame(trend_summary['trend_props'].to_list(), index=trend_summary.index)
        trend_summary = trend_summary.drop(columns=['open', 'high', 'low', 'volume', 'trend_props', 'change'])
        trend_summary.rename(columns={'close_current': 'previous_close', 'close_previous':'day_open', 'change_percentage':'day_change'}, inplace=True)

    else:
        trend_summary_copy = trend_summary.copy()
        trend_summary = pd.merge(trend_summary, current_candles, how = 'inner', on=['token','symbol'], suffixes=('_previous','_current'))
        trend_summary['change'] = trend_summary['close'] - trend_summary['previous_close']
        trend_summary['change_percentage'] = round((trend_summary['change'])*100/trend_summary['previous_close'],2)
        trend_summary['net_change'] = trend_summary['close'] - trend_summary['day_open']
        trend_summary['day_change'] = round((trend_summary['net_change']/trend_summary['day_open'])*100,2)
        trend_summary['trend_props'] = trend_summary.apply(find_positive_trend, axis=1)
        trend_summary[['pause', 'trend', 'start_datetime', 'end_datetime', 'start_price', 'end_price', 'trend_change', 'trend_volume', 'trend_candle_count', 'stag_count', 'half']] = pd.DataFrame(trend_summary['trend_props'].to_list(), index=trend_summary.index)
        trend_summary = trend_summary.drop(columns=['open', 'high', 'low', 'volume', 'trend_props', 'net_change','change_percentage','previous_close','datetime_previous','change'])
        trend_summary.rename(columns={'close': 'previous_close', 'datetime_current': 'datetime'}, inplace=True)

        trend_summary_empty = pd.merge(trend_summary_copy, current_candles, how = 'left', on=['token','symbol'], suffixes=('_current', '_previous'))
        trend_summary_empty = trend_summary_empty[trend_summary_empty['open'].isnull()]
        trend_summary_empty = trend_summary_empty.drop(columns=['open', 'high', 'close', 'low', 'volume', 'datetime_previous'])
        trend_summary_empty.rename(columns={'datetime_current': 'datetime'}, inplace=True)
        if parser.parse(str(distinct_datetime)).time() == datetime.time(9, 16, 0):
            break
        trend_summary = pd.concat([trend_summary, trend_summary_empty], ignore_index=True, sort=True)


    trend_summary['type'] = 'positive'
    trend_summary.to_sql('trend_summary', engine, if_exists='append', index=False)
    



2024-07-12 09:15:00
2024-07-12 09:16:00


In [5]:
trend_summary_empty

,token,symbol,datetime,previous_close,day_open,day_change,half,pause,trend,start_datetime,end_datetime,start_price,end_price,trend_change,trend_volume,trend_candle_count,stag_count,type
42,595,CARBORUNIV,2024-07-12 09:15:00,1695.70,1690.00,0.34,1693.750,False,False,NaT,None,NaN,NaN,NaN,NaN,0,1,positive
61,940,PGHL,2024-07-12 09:15:00,5128.40,5133.20,-0.09,5137.775,False,False,NaT,None,NaN,NaN,NaN,NaN,0,1,positive
206,4898,FDC,2024-07-12 09:15:00,500.95,499.85,0.22,500.475,False,False,NaT,None,NaN,NaN,NaN,NaN,0,1,positive
236,6964,GOCOLORS,2024-07-12 09:15:00,1073.35,1066.00,0.69,1071.725,False,True,2024-07-12 09:15:00,None,1066.00,1073.35,0.69,330.0,1,0,positive
281,10599,GUJGASLTD,2024-07-12 09:15:00,644.20,644.00,0.03,645.175,False,False,NaT,None,NaN,NaN,NaN,NaN,0,1,positive
290,10925,GODREJIND,2024-07-12 09:15:00,911.50,910.00,0.16,910.375,False,False,NaT,None,NaN,NaN,NaN,NaN,0,1,positive
315,11580,WESTLIFE,2024-07-12 09:15:00,857.40,853.00,0.52,853.825,False,True,2024-07-12 09:15:00,None,853.00,857.40,0.52,99.0,1,0,positive
317,11654,LALPATHLAB,2024-07-12 09:15:00,2948.45,2936.10,0.42,2947.175,False,True,2024-07-12 09:15:00,None,2936.10,2948.45,0.42,893.0,1,0,positive
321,11723,JSWSTEEL,2024-07-12 09:15:00,927.40,925.00,0.26,928.375,False,False,NaT,None,NaN,NaN,NaN,NaN,0,1,positive
338,13035,SAFARI,2024-07-12 09:15:00,2159.05,2155.00,0.19,2156.675,False,False,NaT,None,NaN,NaN,NaN,NaN,0,1,positive


In [6]:
trend_summary


,token,symbol,day_open,day_change,half,pause,trend,start_datetime,end_datetime,start_price,end_price,trend_change,trend_volume,trend_candle_count,stag_count,type,datetime,previous_close
0,7,AARTIIND,702.50,0.07,702.700,False,False,NaT,NaT,NaN,NaN,NaN,NaN,0,2,positive,2024-07-12 09:16:00,703.00
1,13,ABB,8470.00,-1.11,8361.700,False,True,2024-07-12 09:16:00,2024-07-12 09:16:00,8347.55,8375.85,0.35,6082.0,1,0,positive,2024-07-12 09:16:00,8375.85
2,22,ACC,2669.25,0.82,2686.950,False,True,2024-07-12 09:15:00,2024-07-12 09:16:00,2669.25,2691.05,0.82,12502.0,2,0,positive,2024-07-12 09:16:00,2691.05
3,25,ADANIENT,3087.00,0.16,3093.825,False,False,NaT,NaT,NaN,NaN,NaN,NaN,0,2,positive,2024-07-12 09:16:00,3091.85
4,40,AEGISCHEM,883.50,1.08,888.900,False,True,2024-07-12 09:16:00,2024-07-12 09:16:00,884.75,893.05,0.97,1617.0,1,0,positive,2024-07-12 09:16:00,893.05
5,48,HAPPSTMNDS,813.85,0.98,821.050,True,True,2024-07-12 09:15:00,NaT,813.85,821.95,1.00,23511.0,1,1,positive,2024-07-12 09:16:00,821.85
6,100,ARE&M,1653.00,-0.85,1637.800,False,False,NaT,NaT,NaN,NaN,NaN,NaN,0,0,positive,2024-07-12 09:16:00,1639.00
7,128,ROUTE,1770.00,0.01,1768.325,False,False,NaT,NaT,NaN,NaN,NaN,NaN,0,2,positive,2024-07-12 09:16:00,1770.20
8,157,APOLLOHOSP,6391.65,-0.37,6372.500,False,False,NaT,NaT,NaN,NaN,NaN,NaN,0,2,positive,2024-07-12 09:16:00,6367.75
9,163,APOLLOTYRE,526.65,-0.41,526.325,False,False,NaT,NaT,NaN,NaN,NaN,NaN,0,2,positive,2024-07-12 09:16:00,524.50
